In [165]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


we need to import the request and access wikipedia

In [166]:
import pandas as pd
import numpy as np
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

import BS for table

In [167]:
from bs4 import BeautifulSoup
bsoup = BeautifulSoup(website_url,'html')
#print(soup.prettify())

In [168]:
My_table = bsoup.find('table',{'class':'wikitable sortable'})
#My_table

here, we need to look at the table, find the body of the table, find the elements between tr and td, create a DF

we reset the index

In [175]:
table_body = My_table.find('tbody')
rows = table_body.find_all('tr')
l=[]
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    l.append(cols)
df_PC=pd.DataFrame(l, columns=["PostalCode","Borough","Neighborhood"])
df_PC=df_PC.drop(df_PC.index[[0]]).reset_index(drop=True)
df_PC.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


now, we need to get rid of not assigned in borough

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [176]:
df_drop = df_PC.drop(df_PC[df_PC.Borough == 'Not assigned'].index).reset_index(drop=True)
df_drop.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


now, we need to replace the neighborood with not assigned

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [177]:
for index, row in df_drop.iterrows():
    if row ['Neighborhood']=='Not assigned':
        row ['Neighborhood']= row ['Borough']
df_drop.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


next, we need to join the neighborood with equal postal code; we group the dt into postal code and borough and we map the values using lambda function

In [178]:
df_final = df_drop.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
df_final.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [174]:
df_final.shape

(103, 3)